In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
from scipy import misc
from PIL import Image

### Creating connection between Python and Sqlite3

In [2]:
conn = sqlite3.connect("local.db")
cursor = conn.cursor()

### Creating Database

In [3]:
cursor.execute("SELECT * FROM invoices LIMIT 5")
query_results_temp = cursor.fetchall()
print(query_results_temp)

[(1, 122, '989319-457', '2014-04-08', 3813.33, 3813.33, 0, 3, '2014-05-08', '2014-05-07'), (2, 123, '263253241', '2014-04-10', 40.2, 40.2, 0, 3, '2014-05-10', '2014-05-14'), (3, 123, '963253234', '2014-04-13', 138.75, 138.75, 0, 3, '2014-05-13', '2014-05-09'), (4, 123, '2-000-2993', '2014-04-16', 144.7, 144.7, 0, 3, '2014-05-16', '2014-05-12'), (5, 123, '963253251', '2014-04-16', 15.5, 15.5, 0, 3, '2014-05-16', '2014-05-11')]


In [4]:
general_ledger_accounts = pd.read_sql(sql="SELECT * FROM general_ledger_accounts",con=conn)
terms = pd.read_sql(sql="SELECT * FROM terms",con=conn)
vendors = pd.read_sql(sql="SELECT * FROM vendors",con=conn)
invoices = pd.read_sql(sql="SELECT * FROM invoices",con=conn)
invoice_line_items = pd.read_sql(sql="SELECT * FROM invoice_line_items",con=conn)
vendor_contacts = pd.read_sql(sql="SELECT * FROM vendor_contacts",con=conn)
invoice_archive = pd.read_sql(sql="SELECT * FROM invoice_archive",con=conn)

# Basic SQL query

### Order of executing SQL query
### FROM - WHERE - GROUP BY - HAVING - SELECT - ORDER BY - LIMIT

In [5]:
# Relationship of the database
im = Image.open('database_relationship.png')
im.show()

### Basic query with `FROM` &`WHERE` & `LIMIT`

In [8]:
cursor.execute("""
SELECT * 
FROM invoices 
WHERE payment_total >10 
LIMIT 2
""")
query_results_temp = cursor.fetchall()
df_temp = pd.DataFrame(query_results_temp)
df_temp.columns=(invoices[(invoices["payment_total"]>10)].columns) # For displaying the column names
df_temp

,invoice_id,vendor_id,invoice_number,invoice_date,invoice_total,payment_total,credit_total,terms_id,invoice_due_date,payment_date
0,1,122,989319-457,2014-04-08,3813.33,3813.33,0,3,2014-05-08,2014-05-07
1,2,123,263253241,2014-04-10,40.20,40.20,0,3,2014-05-10,2014-05-14


In [7]:
invoices[(invoices["payment_total"]>10)].head(2)

,invoice_id,vendor_id,invoice_number,invoice_date,invoice_total,payment_total,credit_total,terms_id,invoice_due_date,payment_date
0,1,122,989319-457,2014-04-08,3813.33,3813.33,0.0,3,2014-05-08,2014-05-07
1,2,123,263253241,2014-04-10,40.20,40.20,0.0,3,2014-05-10,2014-05-14


### Basic query with `WHERE` & `IN` & select specific columns


In [11]:
cursor.execute("""
SELECT 
invoice_id,
invoice_number,
invoice_total
FROM invoices 
WHERE payment_total NOT IN (0,15.5,40.5) 
ORDER BY invoice_id
LIMIT 2
""")
query_results_temp = cursor.fetchall()
df_temp = pd.DataFrame(query_results_temp)
df_temp.columns=(invoices[~invoices["payment_total"].isin([0,15.5,40.5])].sort_values("invoice_id") \
[["invoice_id","invoice_number","invoice_total"]].columns) # For displaying the column names
df_temp

,invoice_id,invoice_number,invoice_total
0,1,989319-457,3813.33
1,2,263253241,40.20


In [12]:
invoices[~invoices["payment_total"].isin([0,15.5,40.5])].sort_values("invoice_id") \
[["invoice_id","invoice_number","invoice_total"]].head(2)

,invoice_id,invoice_number,invoice_total
0,1,989319-457,3813.33
1,2,263253241,40.20


### Basic query with`ORDER BY` using different order `DESC` or `ASC`

In [20]:
cursor.execute("""
SELECT 
invoice_id,
invoice_number,
invoice_total
FROM invoices 
WHERE payment_total > 10
ORDER BY invoice_total, invoice_id DESC
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)

[(59, '4-314-3057', 13.75), (5, '963253251', 15.5)]


In [19]:
invoices[(invoices["payment_total"]>10)].sort_values(by=["invoice_total","invoice_id"],ascending = [True,False]) \
[["invoice_id","invoice_number","invoice_total"]].head(2)


,invoice_id,invoice_number,invoice_total
58,59,4-314-3057,13.75
4,5,963253251,15.50


### Basic query with `WHERE` & `LIMIT` & `ORDER BY` & specific multiple columns

In [8]:
cursor.execute("""
SELECT 
invoice_id,
invoice_number,
invoice_total
FROM invoices 
WHERE payment_total >5 
ORDER BY invoice_id
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)

[(1, '989319-457', 3813.33), (2, '263253241', 40.2)]


In [9]:
invoices[(invoices["payment_total"]>5)].sort_values(by=["invoice_id"]) \
[["invoice_id","invoice_number","invoice_total"]].head(2)

,invoice_id,invoice_number,invoice_total
0,1,989319-457,3813.33
1,2,263253241,40.20


### Basic query with`UNION` & Other functions

In [23]:
cursor.execute("""
SELECT 
invoice_id,
invoice_number,
payment_total
FROM invoices 
WHERE payment_total >50 
UNION 
SELECT 
invoice_id,
invoice_number,
payment_total
FROM invoices 
WHERE payment_total <30 
ORDER BY invoice_id DESC
LIMIT 3
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)

[(114, '963253249', 127.75), (113, '547480102', 0), (112, '0-2436', 0)]


In [31]:
pd.concat([invoices[invoices["payment_total"]>50][["invoice_id","invoice_number","payment_total"]], \
        invoices[invoices["payment_total"]<30][["invoice_id","invoice_number","payment_total"]]]). \
sort_values("invoice_id",ascending = False).head(3)

,invoice_id,invoice_number,payment_total
113,114,963253249,127.75
112,113,547480102,0.00
111,112,0-2436,0.00


# Aggregation SQL query

### Simple aggregation query with `GROUP BY`&`SUM` & `ORDER BY` & `LIMIT`

In [250]:
cursor.execute("""
SELECT  
vendor_id,
SUM(invoice_total)
FROM invoices 
GROUP BY vendor_id,invoice_date
ORDER BY vendor_id
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)


[(34, 116.54), (34, 1083.58)]


In [288]:
invoices.groupby(["vendor_id","invoice_date"]).agg({"invoice_total":"sum"}) \
.reset_index().sort_values(by=["vendor_id"]).head(2)

,vendor_id,invoice_date,invoice_total
0,34,2014-05-07,116.54
1,34,2014-06-09,1083.58


### Simple aggregation query with `SUM` & `ORDER BY` & `LIMIT` & `WHERE` & `HAVING`

In [278]:
cursor.execute("""
SELECT  
vendor_id,
invoice_date,
SUM(invoice_total)
FROM invoices 
WHERE payment_total >500 
GROUP BY vendor_id,invoice_date
HAVING SUM(invoice_total) > 50
ORDER BY vendor_id
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)


[(34, '2014-06-09', 1083.58), (48, '2014-05-03', 856.92)]


In [279]:
temp = \
invoices[(invoices["payment_total"]>500)].groupby(["vendor_id","invoice_date"]) \
.agg({"invoice_total":"sum"}).reset_index()
temp[(temp["invoice_total"]>50)].sort_values(by=["vendor_id"]).head(2)

,vendor_id,invoice_date,invoice_total
0,34,2014-06-09,1083.58
1,48,2014-05-03,856.92


### Aggregation query with `COUNT` & `AVG` for one column & `HAVING` & `WHERE` & select specific columns
#### Select only one aggregation term from the column using `.loc[]`

In [36]:
cursor.execute("""
SELECT  
vendor_id,
invoice_date,
COUNT(invoice_total),
AVG(invoice_total)
FROM invoices 
WHERE payment_total >8 
GROUP BY vendor_id,invoice_date
HAVING COUNT(invoice_total) > 1
ORDER BY COUNT(invoice_total), vendor_id
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)


[(123, '2014-05-31', 2, 226.875), (123, '2014-06-11', 2, 33.5)]


In [39]:
temp = \
invoices[(invoices["payment_total"] > 8)].groupby(["vendor_id","invoice_date"]). \
agg({"invoice_total":['size','mean']}).reset_index(). \
loc[:,[("vendor_id",""),("invoice_date",""),("invoice_total","size"),("invoice_total","mean")]]

temp[temp[("invoice_total","size")]>1].sort_values(by=[("invoice_total","size"),("vendor_id","")]).head(2)


vendor_id invoice_date invoice_total         
                                   size     mean
73       123   2014-05-31             2  226.875
76       123   2014-06-11             2   33.500

# JOIN SQL query

### Simple `INNER` JOIN

In [285]:
cursor.execute("""
SELECT 
v.vendor_name,
i.invoice_date
FROM vendors v
INNER JOIN invoices i ON v.vendor_id = i.vendor_id
ORDER BY v.vendor_name, i.invoice_date
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)


[('Abbey Office Furnishings', '2014-07-05'), ('Bertelsmann Industry Svcs. Inc', '2014-06-18')]


In [286]:
vendors.merge(invoices,how="inner",on="vendor_id").sort_values(by=["vendor_name","invoice_date"]) \
[["vendor_name","invoice_date"]].head(2)

,vendor_name,invoice_date
18,Abbey Office Furnishings,2014-07-05
28,Bertelsmann Industry Svcs. Inc,2014-06-18


### Simple `INNER` & `LEFT` JOIN

In [212]:
cursor.execute("""
SELECT 
v.vendor_name,
i.invoice_date,
i.invoice_total,
ili.line_item_amount
FROM vendors v
INNER JOIN invoices i ON v.vendor_id = i.vendor_id
LEFT JOIN invoice_line_items ili ON i.invoice_id = ili.invoice_id
ORDER BY v.vendor_name, i.invoice_date
LIMIT 2
""")
query_results_temp = cursor.fetchall()
print(query_results_temp)


[('Abbey Office Furnishings', '2014-07-05', 17.5, 17.5), ('Bertelsmann Industry Svcs. Inc', '2014-06-18', 6940.25, 6940.25)]


In [214]:
vendors.merge(invoices,how="inner",on="vendor_id").merge(invoice_line_items,how="left",on="invoice_id") \
[["vendor_name","invoice_date","invoice_total","line_item_amount"]]. \
sort_values(by=["vendor_name","invoice_date"]).head(2)

,vendor_name,invoice_date,invoice_total,line_item_amount
18,Abbey Office Furnishings,2014-07-05,17.50,17.50
31,Bertelsmann Industry Svcs. Inc,2014-06-18,6940.25,6940.25


# Window Function